In [0]:
#imports 

from datetime import datetime
from pyspark.sql.functions import current_timestamp


In [0]:

# Informações Unity Catalog

catalogo =  "fiscaliza_legislativo"
schemaDestino = "silver"
schemaOrigem = "bronze"
tabela = "deputados"

TABELA_SILVER = f"{catalogo}.{schemaDestino}.{tabela}"


# Inicia execução

In [0]:
TInicio = datetime.now()
print(TInicio)

## Criação de Esquema

In [0]:
#Cria Schema Silver

SQL =  f"""
CREATE SCHEMA IF NOT EXISTS {catalogo}.{schemaDestino}
"""
spark.sql(SQL)

In [0]:
#Cria Dataframe da tabela Bronze

df = spark.sql(f"SELECT * FROM {catalogo}.{schemaOrigem}.{tabela}")


In [0]:
# Verifica % de nulos de Nulos

#df.toPandas().isnull().sum() / df.count()

In [0]:
#Colunas a serem removidas
colunas_a_remover = ["email", "uri", "uriPartido", "_data_ingestao"]

#Remove Colunas 
df = df.drop(*colunas_a_remover)

In [0]:
#Inclui data de ingestão na camada silver
df = df.withColumn('_data_ingestao', current_timestamp())
     

In [0]:
#salva Tabela no catalogo

df.write.format("delta") \
              .mode('overwrite') \
              .option("overwriteSchema", "true") \
              .saveAsTable(TABELA_SILVER)

In [0]:
TFim = datetime.now()
print(TFim)
print("Duração:", TFim - TInicio)